<a href="https://colab.research.google.com/github/farsilvar/Image-CNN-FL/blob/main/createImageCpp.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
#!apt-get install -qq gcc-5 g++-5 -y
#!ln -s /usr/bin/gcc-5
#!ln -s /usr/bin/g++-5

#!sudo apt-get update
#!sudo apt-get upgrade

In [ ]:
#!pip install git+https://github.com/andreinechaev/nvcc4jupyter.git

In [ ]:
#%load_ext nvcc_plugin

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
%%writefile createFigure.cpp

#include<vector>
#include<string>
#include<tuple>
#include<limits>
#include<math.h>
#include<iostream>
#include<stdio.h>
#include <fstream>
#include<sstream>
#include<chrono>
#include<bits/stdc++.h>
#include<opencv2/opencv.hpp>
#include<opencv2/highgui.hpp>
#include <sys/stat.h>
#include <sys/types.h>
#include<unistd.h>

using namespace std;
using namespace cv;

class TimeSeries {
	public:
		string rawData;
        bool hasNA;
		vector<double> data;
    vector<double> fts;
    vector<double> pyfts(vector<double> A);
    //vector<<vector<double>> muPyfts(vector<double> A);
		int label;
		vector<double> nfn();
    vector<double> createFTS();
		tuple<double, double, int, int, double, double> y_nfn_func(double x,int n,int m,double delta,vector<double> b,double xmin,double xmax,vector<double> w);

		TimeSeries(string raw){
			rawData = raw;
			string inString = rawData.substr(0,rawData.find(':'));
            hasNA = false;
            try {
			    while (inString.find(',') != std::string::npos) {
			    	data.push_back(stold(inString.substr(0,inString.find(','))));
			    	inString.erase(0,inString.find(',') + 1);
			    }
			    data.push_back(stold(inString));
			    label = stoi(rawData.erase(0,rawData.find(':') + 1));
                //throw ("std::invalid_argument");
            }
            catch(const std::invalid_argument& e){
                //std::cerr << "Error: Invalid value in file (" << e.what() << "): '" << inString << "'" << std::endl;
                hasNA = true;
            }
			rawData = raw;
            fts = createFTS();
		};
        double triangular(double x, double a, double b, double c){
            // Finding the largest of all the numbers
            double mMin = (double)((x-a)/(b-a) < (c-x)/(c-b)) ? (x-a)/(b-a) : (c-x)/(c-b);
            double mMax = (double)(mMin > 0) ? mMin : 0;
            return mMax;
        }
		vector<double> fuzzify(vector<vector<double>> xfuzzy){
			vector<double> xFTS (xfuzzy.size());
        	for (unsigned k = 0; k < xfuzzy.size(); k++){
				vector<double> kvec = xfuzzy[k];
				double max {}; //= *max_element(kvec.begin(), kvec.end());
				for (unsigned l = 0; l < xfuzzy[k].size(); l++){
					if (xfuzzy[k].at(l) >= max){
						max = xfuzzy[k].at(l);
						xFTS[k] = l; // find(kvec.begin(), kvec.end(), max);
					}
				}

			}

        	return xFTS;

		};
};

vector<double> TimeSeries::createFTS(){

    double xmin {numeric_limits<double>::max()}; // = min_element(x_train.begin(),x_train.end()) - x_train.begin();
    double xmax {}; // = max_element(x_train.begin(), x_train.end()) - x_train.begin();
	for (unsigned i = 0; i < data.size(); i++){
	    if (data.at(i) <= xmin){
	    	xmin = data.at(i);
		} else if (data.at(i) >= xmax){
			xmax = data.at(i);
		}
	}
    double maxValue = 1.2*xmax;
    double minValue = 0.8*xmin;

    int nGroups = 10;

    double A[nGroups + 2] = {0};
    double partLen = (xmax - xmin)/(nGroups);
    A[0] = xmin - partLen/2;
    A[nGroups + 1] = xmax + partLen/2;
    for (int i = 1; i < nGroups+1; i++){
        A[i] = A[i-1] + partLen;
    }

    vector<double> fuzzy(data.size());
    int j = 0;
    for (unsigned i = 0; i < data.size(); i++){
        int fuzzyCalc = 0;
        for (int u = 1; u < nGroups + 1; u++){
            double calc = triangular(data.at(i), A[u-1], A[u], A[u+1]);
            if (calc > fuzzyCalc){
                fuzzyCalc = calc;
                fuzzy[j] = u-1;
            }
        }
        j++;
    }

    return fuzzy;
}

vector<double> TimeSeries::pyfts(vector<double> A){

    int nGroups = 10;

    vector<double> fuzzy(data.size());
    int j = 0;
    for (unsigned i = 0; i < data.size(); i++){
        double fuzzyCalc = 0;
        for (int u = 1; u < nGroups + 1; u++){
            double calc = triangular(data.at(i), A[u-1], A[u], A[u+1]);
            if (calc > fuzzyCalc){
                fuzzyCalc = calc;
                fuzzy[j] = u-1;
            }
        }
        j++;
    }

    return fuzzy;
}
/*
vector<vector<double>> TimeSeries::muPyfts(vector<double> A){

    int nGroups = 10;

    vector<double> fuzzyRow(nGroups);
    vector<<vector<double>> fuzzy(data.size(), fuzzyRow);

    int j = 0;
    for (int u = 1; u < nGroups + 1; u++){
        for (unsigned i = 0; i < data.size(); i++){
            double fuzzyCalc = 0;
            double calc = triangular(data.at(i), A[u-1], A[u], A[u+1]);
            fuzzy[u-1][j] = calc;
            j++;
        }
    }

    return fuzzy;
} */
vector<double> TimeSeries::nfn(){

        const int m {10};
        vector<double> x_train = data;
        const int np_train = x_train.size();
        vector<double> yd = x_train;
        const int n {1}; //np.shape(x_train)[1]
        float alpha = 0.01;
        int maxEpocas = 1;

        double xmin {numeric_limits<double>::max()}; // = min_element(x_train.begin(),x_train.end()) - x_train.begin();
        double xmax {}; // = max_element(x_train.begin(), x_train.end()) - x_train.begin();
		for (unsigned i = 0; i < x_train.size(); i++){
			if (x_train.at(i) <= xmin){
				xmin = x_train.at(i);
			} else if (x_train.at(i) >= xmax){
				xmax = x_train.at(i);
			}
		}

		vector<double> b;
		vector<double> w;
		double ys,yi {};
		int k1, k2;
		double muk1, muk2;

         // Inicializar os parâmetros b, w e delta

        double delta = (xmax - xmin)/(m-1);
        for (auto j = 0; j < m; j++){
            b.push_back(xmin + (j)*delta);
			w.push_back(rand());
		}

        double ei2 = 0;
        int epoca = 0;
        vector<double> erroEpoca;
	    erroEpoca.push_back(numeric_limits<double>::max());
        vector<vector<double>> xfuzzy(np_train,vector<double>(m));

        while (erroEpoca[epoca] > 1e-5 and epoca <= maxEpocas){
            for (auto k = 0; k < np_train; k++){
                double xin = x_train.at(k);
                tie(ys, yi, k1, k2, muk1, muk2) = y_nfn_func(xin, n, m, delta, b, xmin, xmax, w);
                double dedys = ys - yd.at(k);

                w[k1] = w[k1] - alpha*dedys*muk1;
                w[k2] = w[k2] - alpha*dedys*muk2;
                xfuzzy[k][k1] = muk1;
                xfuzzy[k][k2] = muk2;

                ei2 = pow((ys - yd[k]),2);
                ei2 = ei2 + ei2;
			}
            epoca += 1;
            erroEpoca.push_back(ei2);
		}

/*
    for (unsigned i = 0; i < xfuzzy.size(); i++){
      for (unsigned j = 0; j < xfuzzy[i].size(); j++){
        cout << xfuzzy[i][j] << ",";
      }
      cout << endl;
    }
*/
		vector<double> xFTS = fuzzify(xfuzzy);
		return xFTS;
}

tuple<double, double, int, int, double, double> TimeSeries::y_nfn_func(double x,int n,int m,double delta,vector<double> b,double xmin,double xmax,vector<double> w){

	int k1 {};
	double muk1 {};
    int k2 {};
    double muk2 {};
	double yi {};

    if (x <= xmin){
		k1 = 0;
        muk1 = 1;
    }else if (x >= xmax){
        k1 = m - 2;
        muk1 = 0;
    }else {
        k1 = int((x - xmin)/delta);
        muk1 = 1 - (x - b[k1])/delta;
	}

	k2 = k1 + 1;
	muk2 = 1 - muk1;
    yi = muk1*w[k1] + muk2*w[k2];

	double ys = yi;

    return make_tuple(ys, yi, k1, k2, muk1, muk2);

}

//int ** matrixImage(vector<double> timeSerie){

void matrixImage(vector<double> timeSerie, string dataSetType, int count, int label, string myDir){

    //int j = 0;
    vector<double> sortMatrix = timeSerie;
    vector<int> tsMatch;
    sort(sortMatrix.begin(), sortMatrix.end());
    unsigned h = timeSerie.size();
    unsigned w = timeSerie.size();
    uint8_t indexColumn[h][w] = {0};
    //int **indexColumn = 0;
    //indexColumn = new int*[h];
    for (int i = 0; i<h; i++){
        //indexColumn[i] = new int[w];
        for(int j = 0; j < w; j++){
            indexColumn[i][j] = 255;
            if (sortMatrix[i] == timeSerie[j]){
                //tsMatch.push_back(j);
                indexColumn[i][j] = 0;
            }
        }
    }
    //    indexColumn[tsmatch,i] = 1 #np.count_nonzero(timeSerie == timeSerie[i])

    Mat imgGray(timeSerie.size(), timeSerie.size(), CV_8UC1, indexColumn);
    Size size(300,300);//the dst image size,e.g.100x100
    Mat dst;//dst image
    //resize(imgGray,dst,size);//resize image

    //string myDir = get_current_dir_name();
    string classLabel = to_string(label);
    //string dataSetType = "TestSet";
    string myFileName = "ts_" + to_string(count) + ".png";
    string myPath = myDir + "/" + dataSetType + "/" + classLabel + "/" + myFileName;
    struct stat buffer;
    string imgPath = dataSetType + "/" + classLabel;
    //mkdir("testSet/TesteNovo", 0777);

    // Checa se diretório existe e cria diretório
    if (stat((myDir + "/" + imgPath).c_str(), &buffer) != 0){
        mkdir((myDir + "/" + dataSetType).c_str(), 0777);
        mkdir((myDir + "/" + imgPath).c_str(), 0777);

        //system(("mkdir -p " + imgPath).c_str());
    }

    bool check = imwrite(myPath, imgGray);

    if(!check){
        cout << "Error!!! Couldn't create image!" << endl;
    }

    //namedWindow("test", cv::WINDOW_AUTOSIZE);
    //imshow("test", dst);
    //waitKey(0);

    // destroying the creating window
    //destroyWindow("test");
}

/*
void matrixImage(vector<vector<double>> timeSerieMatrix, string dataSetType, int count, int label, string myDir){

    //int j = 0;
    vector<vector<double>> sortMatrix = timeSerieMatrix;
    vector<int> tsMatch;
    sort(sortMatrix.begin(), sortMatrix.end());
    unsigned h = timeSerie.size();
    unsigned w = timeSerie.at(0).size();
    uint8_t indexColumn[h][w] = {0};
    //int **indexColumn = 0;
    //indexColumn = new int*[h];
    for (int i = 0; i<h; i++){
        //indexColumn[i] = new int[w];
        for(int j = 0; j < w; j++){
            indexColumn[i][j] *= 255;
        }
    }
    //    indexColumn[tsmatch,i] = 1 #np.count_nonzero(timeSerie == timeSerie[i])

    Mat imgGray(timeSerie.size(), timeSerie.at(0).size(), CV_8UC1, indexColumn);
    Size size(300,300);//the dst image size,e.g.100x100
    Mat dst;//dst image
    //resize(imgGray,dst,size);//resize image

    //string myDir = get_current_dir_name();
    string classLabel = to_string(label);
    //string dataSetType = "TestSet";
    string myFileName = "ts_" + to_string(count) + ".png";
    string myPath = myDir + "/" + dataSetType + "/" + classLabel + "/" + myFileName;
    struct stat buffer;
    string imgPath = dataSetType + "/" + classLabel;
    //mkdir("testSet/TesteNovo", 0777);

    // Checa se diretório existe e cria diretório
    if (stat((myDir + "/" + imgPath).c_str(), &buffer) != 0){
        mkdir((myDir + "/" + dataSetType).c_str(), 0777);
        mkdir((myDir + "/" + imgPath).c_str(), 0777);

        //system(("mkdir -p " + imgPath).c_str());
    }

    bool check = imwrite(myPath, imgGray);

    if(!check){
        cout << "Error!!! Couldn't create image!" << endl;
    }

    //namedWindow("test", cv::WINDOW_AUTOSIZE);
    //imshow("test", dst);
    //waitKey(0);

    // destroying the creating window
    //destroyWindow("test");
} */

    //indexColumn /= np.max(indexColumn)


int main(int argc, char* argv[])
{
	auto begin = std::chrono::high_resolution_clock::now();
  string dataName = argv[1];
  string partitionerType = argv[2];
  string path_to_data = "drive/My Drive/Doutorado/Regular/modelo/UCR/";
  string totalPath = path_to_data + dataName;
  ifstream trainFile {totalPath + "/" + dataName + "_TRAIN.ts"}; // ‘‘i’’ for ‘‘input’’
  ifstream testFile {totalPath + "/" + dataName + "_TEST.ts"}; // ‘‘i’’ for ‘‘input’’
  if (!trainFile){
    cerr << "error: unable to open input file!\n";
    return -1;
	}

  if (!testFile){
    cerr << "error: unable to open input file!\n";
    return -1;
	}

  string word;
	vector<TimeSeries> trainTS;
	vector<TimeSeries> testTS;

    bool isData = 0;
    while (getline (trainFile, word)) {
  	// Output the text from the file
		if (!isData){
            if (word == "@data"){
                isData = 1;
            }
            continue;
        }
        if (isData){
            trainTS.push_back(word);
        }
	}

    isData = 0;
    while (getline (testFile, word)) {
  	 //Output the text from the file
		if (!isData){
            if (word == "@data"){
                isData = 1;
            }
            continue;
        }
        if (isData){
		    testTS.push_back(word);
        }
	}

  ifstream A_trainFile {totalPath + "/" + dataName + "Train_" + partitionerType + ".csv"}; // ‘‘i’’ for ‘‘input’’
  ifstream A_testFile {totalPath + "/" + dataName + "Test_"  + partitionerType + ".csv"}; // ‘‘i’’ for ‘‘input’’

  vector<vector<double>> A_train;
  vector<vector<double>> A_test;

  if (!A_trainFile){
    cerr << "error: unable to open train partition!\n";
    return -1;
	}

  if (!A_testFile){
    cerr << "error: unable to open test partition!\n";
    return -1;
	}

  string wordA;
  vector<string> partTrain;
  vector<string> partTest;

  while (getline (A_trainFile, wordA)) {
  	// Output the text from the file
      vector<double> A_col;
		  partTrain.push_back(wordA);
      while (wordA.find(',') != std::string::npos) {
			  A_col.push_back(stold(wordA.substr(0,wordA.find(','))));
				wordA.erase(0,wordA.find(',') + 1);
			}
      A_col.push_back(stold(wordA));
      A_train.push_back(A_col);
	}

  while (getline (A_testFile, wordA)) {
  	// Output the text from the file
      vector<double> A_col;
		  partTest.push_back(wordA);
      while (wordA.find(',') != std::string::npos) {
			  A_col.push_back(stold(wordA.substr(0,wordA.find(','))));
				wordA.erase(0,wordA.find(',') + 1);
			}
      A_col.push_back(stold(wordA));
      A_test.push_back(A_col);
	}

    //trainTS[k].data = {4,2,3,4};
    //int** s = matrixImage(trainTS[k].data); //trainTS[k].data);

//    for(unsigned i = 0; i < trainTS[0].fts().size(); i++){
//        cout << trainTS[10].fts().at(i) << endl;
//    }

    string tsTransform = argv[3];

    cout << "Imagens do banco de dados " << dataName << " na forma " << tsTransform << ".\n\n";

    for (int k = 0; k < trainTS.size(); k++){
        cout << "Criando imagem de treinamento " + to_string(k + 1) + " / " + to_string(trainTS.size()) << endl;
        if (tsTransform == "RAW"){
            if (trainTS[k].hasNA){
                cout << "Has NA" << endl;
                continue;
            }
            matrixImage(trainTS[k].data, "trainDataCppPaper" + tsTransform, k+1, trainTS[k].label, totalPath);
        } else if (tsTransform == "FTS"){
          matrixImage(trainTS[k].fts, "trainDataCppPaper" + tsTransform, k+1, trainTS[k].label, totalPath);
        } else if (tsTransform == "PYFTS"){
            if (trainTS[k].hasNA){
                cout << "Has NA" << endl;
                continue;
            }
          matrixImage(trainTS[k].pyfts(A_train[k]), "trainDataCppPaper" + partitionerType + tsTransform, k+1, trainTS[k].label, totalPath);
        } else if (tsTransform == "NFN"){
          matrixImage(trainTS[k].nfn(), "trainDataCppPaper" + tsTransform, k+1, trainTS[k].label, totalPath);
        }
    }

    for (int k = 0; k < testTS.size(); k++){
        cout << "Criando imagem de teste " + to_string(k + 1) + " / " + to_string(testTS.size()) << endl;
        if (tsTransform == "RAW"){
            if (testTS[k].hasNA){
                cout << "Has NA" << endl;
                continue;
            }
          matrixImage(testTS[k].data, "testDataCppPaper" + tsTransform, k+1, testTS[k].label, totalPath);
        } else if (tsTransform == "FTS"){
          matrixImage(testTS[k].fts, "testDataCppPaper" + tsTransform, k+1, testTS[k].label, totalPath);
        } else if (tsTransform == "PYFTS"){
            if (testTS[k].hasNA){
                cout << "Has NA" << endl;
                continue;
            }
          matrixImage(testTS[k].pyfts(A_test[k]), "testDataCppPaper" + partitionerType + tsTransform, k+1, testTS[k].label, totalPath);
        //} else if (tsTransform == "MUPYFTS"){
        //  matrixImage(testTS[k].muPyfts(A_test[k]), "testDataCpp" + tsTransform, k+1, testTS[k].label, totalPath);
        } else if (tsTransform == "NFN"){
          matrixImage(testTS[k].nfn(), "testDataCppPaper" + tsTransform, k+1, testTS[k].label, totalPath);
        }

    }

    //int k = 0;
    //matrixImage(testTS[k].data, "testDataCppThesis", k+1, testTS[k].label, totalPath);
    //system("mkdir -p testSet/TesteNovo");
    /*
    // important: clean up memory
    printf("\n");
    printf("Cleaning up memory...\n");
    for (int h = 0; h < trainTS[k].data.size(); h++) // loop variable wasn't declared
    {
        delete [] s[h];
    }
    delete [] s;
    s = 0;
    printf("Ready.\n");
    */

	trainFile.close();
	testFile.close();
  A_trainFile.close();
  A_testFile.close();

	auto end = std::chrono::high_resolution_clock::now();
    auto elapsed = std::chrono::duration_cast<std::chrono::nanoseconds>(end - begin);

    cout << "Elapsed time: " << elapsed.count()*1e-9 << endl;


	return 0;
}

Writing createFigure.cpp


In [ ]:
%%bash
g++ createFigure.cpp -o createFigure `pkg-config --cflags --libs opencv4`

In [ ]:
#Dataset Partition tsTransform[RAW FTS PYFTS NFN]
!./createFigure DodgerLoopGame Grid RAW

Imagens do banco de dados DodgerLoopGame na forma RAW.

Criando imagem de treinamento 1 / 20
Criando imagem de treinamento 2 / 20
Criando imagem de treinamento 3 / 20
Criando imagem de treinamento 4 / 20
Criando imagem de treinamento 5 / 20
Criando imagem de treinamento 6 / 20
Criando imagem de treinamento 7 / 20
Criando imagem de treinamento 8 / 20
Has NA
Criando imagem de treinamento 9 / 20
Criando imagem de treinamento 10 / 20
Criando imagem de treinamento 11 / 20
Criando imagem de treinamento 12 / 20
Criando imagem de treinamento 13 / 20
Criando imagem de treinamento 14 / 20
Criando imagem de treinamento 15 / 20
Criando imagem de treinamento 16 / 20
Has NA
Criando imagem de treinamento 17 / 20
Criando imagem de treinamento 18 / 20
Criando imagem de treinamento 19 / 20
Criando imagem de treinamento 20 / 20
Has NA
Criando imagem de teste 1 / 138
Criando imagem de teste 2 / 138
Criando imagem de teste 3 / 138
Criando imagem de teste 4 / 138
Criando imagem de teste 5 / 138
Criando imag

In [ ]:
#Dataset Partition tsTransform[RAW FTS PYFTS NFN]
!./createFigure DodgerLoopGame Grid PYFTS

Imagens do banco de dados DodgerLoopGame na forma PYFTS.

Criando imagem de treinamento 1 / 20
Criando imagem de treinamento 2 / 20
Criando imagem de treinamento 3 / 20
Criando imagem de treinamento 4 / 20
Criando imagem de treinamento 5 / 20
Criando imagem de treinamento 6 / 20
Criando imagem de treinamento 7 / 20
Criando imagem de treinamento 8 / 20
Has NA
Criando imagem de treinamento 9 / 20
Criando imagem de treinamento 10 / 20
Criando imagem de treinamento 11 / 20
Criando imagem de treinamento 12 / 20
Criando imagem de treinamento 13 / 20
Criando imagem de treinamento 14 / 20
Criando imagem de treinamento 15 / 20
Criando imagem de treinamento 16 / 20
Has NA
Criando imagem de treinamento 17 / 20
Criando imagem de treinamento 18 / 20
Criando imagem de treinamento 19 / 20
Criando imagem de treinamento 20 / 20
Has NA
Criando imagem de teste 1 / 138
Criando imagem de teste 2 / 138
Criando imagem de teste 3 / 138
Criando imagem de teste 4 / 138
Criando imagem de teste 5 / 138
Criando im

In [ ]:
#Dataset Partition tsTransform[RAW FTS PYFTS NFN]
!./createFigure FordB CMeans PYFTS

error: unable to open train partition!


In [ ]:
#Dataset Partition tsTransform[RAW FTS PYFTS NFN]
!./createFigure FordB Grid NFN

Imagens do banco de dados FordB na forma NFN.

Criando imagem de treinamento 1 / 3636
Criando imagem de treinamento 2 / 3636
Criando imagem de treinamento 3 / 3636
Criando imagem de treinamento 4 / 3636
Criando imagem de treinamento 5 / 3636
Criando imagem de treinamento 6 / 3636
Criando imagem de treinamento 7 / 3636
Criando imagem de treinamento 8 / 3636
Criando imagem de treinamento 9 / 3636
Criando imagem de treinamento 10 / 3636
Criando imagem de treinamento 11 / 3636
Criando imagem de treinamento 12 / 3636
Criando imagem de treinamento 13 / 3636
Criando imagem de treinamento 14 / 3636
Criando imagem de treinamento 15 / 3636
Criando imagem de treinamento 16 / 3636
Criando imagem de treinamento 17 / 3636
Criando imagem de treinamento 18 / 3636
Criando imagem de treinamento 19 / 3636
Criando imagem de treinamento 20 / 3636
Criando imagem de treinamento 21 / 3636
Criando imagem de treinamento 22 / 3636
Criando imagem de treinamento 23 / 3636
Criando imagem de treinamento 24 / 3636
Cr